In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
lark_dw = pd.read_csv("./lark_job.csv")
# lark_dw.isnull().sum()

In [ ]:
lark_dw.fillna(0, inplace=True)
# lark_dw.head(10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# lark_dw["fuxi_task_num"].value_counts()
#lark_dw[lark_dw["fuxi_task_num"] == 53]
lark_dw.columns

In [ ]:
lark_dw[lark_dw["odps_inst_id"] == "20220206035950687grs6g5ys"].fuxi_inst_num

In [ ]:
task_num = lark_dw["fuxi_task_num"].values
project_name = lark_dw["project_name"].values
project_map = {}
index = 0
for i in range(len(project_name)):
    if project_name[i] not in project_map:
        project_map[project_name[i]] = index
        index += 1
    project_name[i] = project_map[project_name[i]]

inst_num = lark_dw["fuxi_inst_num"].values
cpu_cost = lark_dw["fuxi_cpu_cost"].values
mem_cost = lark_dw["fuxi_mem_cost"].values
start_time = (lark_dw["start_time"].values % 86400)
end_time = (lark_dw["end_time"].values % 86400)
job_type = lark_dw["fuxi_job_sn"].values
quota_id = lark_dw["fuxi_quota_id"].values
plan_cpu = lark_dw["plan_cpu_cost"].values
plan_mem = lark_dw["plan_mem_cost"].values
duration = []
for i in range(start_time.shape[0]):
    if end_time[i] > start_time[i]:
        duration.append(end_time[i] - start_time[i])
    else:
        duration.append(end_time[i] + 86400 - start_time[i])
duration = np.asarray(duration)
X = np.vstack((inst_num, cpu_cost, mem_cost, start_time, end_time, duration, plan_cpu, plan_mem, job_type, quota_id, project_name, task_num)).T
X.shape

In [ ]:
task_type = len(list(lark_dw["fuxi_task_num"].value_counts()))
lark_dw["fuxi_task_num"].value_counts()

In [ ]:
model = RandomForestClassifier()
X_train,X_test,y_train,y_test = train_test_split(X[:, [0, 1, 3, 6, 7, 8, 9, 10]], task_num, test_size=0.1, random_state=0)
X_train.shape
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
accuracy = [0 for i in range(100)]
total = [0 for i in range(100)]
y_predict = model.predict(X_test)
for i in range(y_predict.shape[0]):
    total[y_test[i]] += 1
    if y_predict[i] == y_test[i]:
        accuracy[y_test[i]] += 1
for i in range(100):
    if accuracy[i]:
        print(i, accuracy[i], accuracy[i]/total[i])

## cpu_cost bucket

In [ ]:
avg_cpu_cost = cpu_cost / inst_num
label = [0 for i in range(avg_cpu_cost.shape[0])]
# avg_cpu_cost = sorted(avg_cpu_cost)
# y = 100. * np.arange(len(avg_cpu_cost))/(len(avg_cpu_cost)-1)
# plt.figure()
# plt.plot(avg_cpu_cost, y)
# plt.xlim(0, 2000)
for i in range(avg_cpu_cost.shape[0]):
    if avg_cpu_cost[i] < 200:
        label[i] = int(avg_cpu_cost[i]/20)
    elif avg_cpu_cost[i] < 600:
        label[i] = int(avg_cpu_cost[i]/50 + 6)
    elif avg_cpu_cost[i] < 2000:
        label[i] = int(avg_cpu_cost[i]/200 + 15)
    elif avg_cpu_cost[i] >= 10000:
        label[i] = 25
        

In [ ]:
model = RandomForestClassifier()
X_train,X_test,y_train,y_test = train_test_split(X[:, [0, 3, 4, 5, 6, 7, 8, 9, 10, 11]], label, test_size=0.1, random_state=0)
X_train.shape
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
accuracy = [0 for i in range(100)]
ambiguous_accuracy = [0 for i in range(100)]
total = [0 for i in range(100)]
y_predict = model.predict(X_test)
for i in range(y_predict.shape[0]):
    total[y_test[i]] += 1
    if y_predict[i] == y_test[i]:
        accuracy[y_test[i]] += 1
    if abs(y_predict[i] - y_test[i]) <= 1:
        ambiguous_accuracy[y_test[i]] += 1

for i in range(100):
    if accuracy[i]:
        print(i, accuracy[i], accuracy[i]/total[i], ambiguous_accuracy[i]/total[i])

## duration bucket

In [ ]:
label = [0 for i in range(len(duration))]
# duration = sorted(duration)
# y = 100. * np.arange(len(duration))/(len(duration)-1)
# plt.figure()
# plt.plot(duration, y)
# plt.xlim(0, 400)
for i in range(len(duration)):
    if duration[i] < 50:
        label[i] = int(duration[i]/5)
    elif avg_cpu_cost[i] < 100:
        label[i] = int(avg_cpu_cost[i]/10 + 5)
    elif avg_cpu_cost[i] < 200:
        label[i] = int(avg_cpu_cost[i]/20 + 10)
    elif avg_cpu_cost[i] >= 400:
        label[i] = 20
        

In [ ]:
model = RandomForestClassifier()
X_train,X_test,y_train,y_test = train_test_split(X[:, [0, 3, 6, 7, 8, 9, 10, 11]], label, test_size=0.1, random_state=0)
X_train.shape
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
accuracy = [0 for i in range(100)]
ambiguous_accuracy = [0 for i in range(100)]
total = [0 for i in range(100)]
y_predict = model.predict(X_test)
for i in range(y_predict.shape[0]):
    total[y_test[i]] += 1
    if y_predict[i] == y_test[i]:
        accuracy[y_test[i]] += 1
    if abs(y_predict[i] - y_test[i]) <= 4:
        ambiguous_accuracy[y_test[i]] += 1

for i in range(100):
    if accuracy[i]:
        print(i, accuracy[i], accuracy[i]/total[i], ambiguous_accuracy[i]/total[i])

In [ ]:
category_cpu = [[] for i in range(100)]
category_instance = [[] for i in range(100)]
for i in range(task_num.shape[0]):
    category_cpu[task_num[i]].append(cpu_cost[i])
    category_instance[task_num[i]].append(inst_num[i])


In [ ]:
plt.figure()
plt.scatter(category_cpu[1], category_instance[1])
plt.xscale("log")
plt.yscale("log")
plt.show()

全特征 精度 1/0.931
0,0.7072580214459075,0.703497997329773
1,0.7630387438656041,0.47540720961281707
2,0.9110416159793967,0.3983444592790387
3,0.6765726899956681,0.4436849132176235
4,0.6768219230107348,0.4200267022696929
5,0.4709673208044292,0.4597596795727637
6,0.728822611368586,0.5590921228304406
7,0.6989680566352356,0.5451001335113485
8,0.40302758772112013,0.4022429906542056
9,0.36057489748808724,0.36352469959946593
instance数量，使用cpu，开始时间，0.9999940658805937, 0.8446461949265688
instance数量，使用cpu，开始时间，结束时间，0.9999762635223746, 0.8840587449933245
除去使用mem，结束时间0.9999881317611873, 0.9154606141522029

In [ ]:
X = np.vstack((inst_num, task_num, start_time, project_name)).T
# X

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
X_train,X_test,y_train,y_test = train_test_split(X, cpu_cost, test_size=0.1, random_state=0)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
y_train_predict = model.predict(X_train)
plt.figure(figsize=(20,10))
plt.plot(range(200), y_train[0:200])
plt.plot(range(200), y_train_predict[0:200])
plt.legend(["train", "predict"])
plt.yscale("log")
plt.show()

y_test_predict = model.predict(X_test)
plt.figure(figsize=(20,10))
plt.plot(range(200), y_test[0:200])
plt.plot(range(200), y_test_predict[0:200])
plt.legend(["test", "predict"])
plt.yscale("log")
plt.show()

In [ ]:
error = []
for i in range(y_train.shape[0]):
    if y_train[i] == 0:
        error.append(y_train_predict[i])
    else:
        error.append((y_train[i] - y_train_predict[i]) / y_train[i])
error_interval = [0 for i in range(11)]
for i in error:
    index = int(i//0.2+5)
    if index > 10:
        index = 10
    elif index < 0:
        index = 0
    error_interval[index] += 1
train_error_interval = np.asarray(error_interval) / len(error)
error = []
for i in range(y_test.shape[0]):
    if y_test[i] == 0:
        error.append(y_test_predict[i])
    else:
        error.append((y_test[i] - y_test_predict[i]) / y_test[i])
error_interval = [0 for i in range(11)]
for i in error:
    index = int(i//0.2+5)
    if index > 10:
        index = 10
    elif index < 0:
        index = 0
    error_interval[index] += 1
test_error_interval = np.asarray(error_interval) / len(error)

In [ ]:
plt.figure()
plt.plot(range(11), train_error_interval)
plt.plot(range(11), test_error_interval)

In [ ]:
error = []
for i in range(y_test.shape[0]):
    if y_test[i] == 0:
        error.append(abs(y_test_predict[i]))
    else:
        error.append(abs((y_test[i] - y_test_predict[i]) / y_test[i]))
error = sorted(error)
train_error = 100. * np.arange(len(error))/(len(error)-1)
plt.figure()
plt.plot(error, train_error)
plt.xlim(0, 1)

In [ ]:
model.predict(X_test)[0:30]

instance_num, task_num, start_time作为特征
预测duration: 0.583453995731672, 0.14398273766226943
预测cpu_cost: 0.9973088839129767, 0.9888808900851879
增加job_type, quota_id, plan_cpu, plan_mem
duration: 0.6569621694905388, 0.22182187327722058
cpu_cost: 0.9971900615327868, 0.9888324466271863
和instance_num的相关性最大

任务提交时，已知task_num, instance_num, start_time
可以预测总CPU用时（属于任务的所有实例的CPU运行时间）
还需区分不同task（CPU用量不同），预测运行时间
可以从instance的数据中得到（但是误差较大）

In [ ]:
lark_dw.drop(columns=["running_cluster","real_cpu_cost", "real_mem_cost"], inplace=True)

In [ ]:
lark_dw[lark_dw["fuxi_job_name"]=="lark_dw_20220206035950687grs6g5ys_SQL_0_1_0_job_0"]

In [ ]:
lark_dw_instance = pd.read_csv("./lark_instance.csv")

In [ ]:
lark_dw_instance.drop(columns=["total_seq_no", "seq_no"], inplace=True)
lark_dw_instance.fillna(0, inplace=True)

In [ ]:
target = lark_dw_instance[lark_dw_instance["odps_inst_id"]=="20220206040005867gizk70m"]
target.shape
# target[target["fuxi_task_name"] != "M1"].head(50)
# target.sort_values("worker_address", inplace=True)
# target.to_csv("./tmp.csv")

In [ ]:
cpu_usage = target["fuxi_avg_cpu"] * (target["end_time_ms"]-target["start_time_ms"]) /1000


In [ ]:
n, bins, patches = plt.hist(cpu_usage, 50, density=True, edgecolor="black", facecolor="royalblue")

In [ ]:
# lark_dw_instance.sort_values("fuxi_avg_cpu", inplace=True)
avg_cpu = lark_dw_instance["fuxi_avg_cpu"].values
np.percentile(avg_cpu, 85)

In [ ]:
cpu_usage = (target["fuxi_avg_cpu"] * (target["end_time_ms"]-target["start_time_ms"])).values / 1000
task_name = target["fuxi_task_name"].values
mp = {}
for i in range(task_name.shape[0]):
    if task_name[i] not in mp:
        mp[task_name[i]] = []
    mp[task_name[i]].append(cpu_usage[i])

In [ ]:
for key in mp:
    print(key, len(mp[key]))
    if(len(mp[key])<2):
        continue
    tmp = sorted(mp[key])
    y = 100. * np.arange(len(tmp))/(len(tmp)-1)
    plt.figure()
    plt.plot(tmp, y)
    plt.show()
#     plt.xlim(0, 1)